In [1]:
import utils

In [40]:
import numpy as np

class PoseSequence:
    def __init__(self, sequence):
        self.poses = []
        for parts in sequence:
            self.poses.append(Pose(parts))
        # normalize poses based on the average torso pixel length
        torso_lengths = np.array([Part.dist(pose.lshoulder, pose.lhip) for pose in self.poses if pose.lshoulder.exists and pose.lhip.exists] +
                                 [Part.dist(pose.rshoulder, pose.rhip) for pose in self.poses if pose.rshoulder.exists and pose.rhip.exists])
        mean_torso = np.mean(torso_lengths)
        print(torso_lengths)
        for pose in self.poses:
            for attr, part in pose:
#                 print(attr, part.x)
                setattr(pose, attr, part / mean_torso)



class Pose:
    PART_NAMES = ['nose', 'neck',  'rshoulder', 'relbow', 'rwrist', 'lshoulder', 'lelbow', 'lwrist', 'rhip', 'rknee', 'rankle', 'lhip', 'lknee', 'lankle', 'reye', 'leye', 'rear', 'lear']

    def __init__(self, parts):
        """Construct a pose for one frame, given an array of parts

        Arguments:
            parts - 18 * 3 ndarray of x, y, confidence values
        """
#         print("Start")
        for name, vals in zip(self.PART_NAMES, parts):
#             print(name, Part(vals).x)
            setattr(self, name, Part(vals))
    
    def __iter__(self):
        for attr, value in self.__dict__.items():
            yield attr, value
    
    def __str__(self):
        out = ""
        for name in self.PART_NAMES:
            _ = "{}: {},{}".format(name, getattr(self, name).x, getattr(self, name).x)
            out = out + _ +"\n"
        return out
    
    def print(self, parts):
        out = ""
        for name in parts:
            if not name in self.PART_NAMES:
                raise NameError(name)
            _ = "{}: {},{}".format(name, getattr(self, name).x, getattr(self, name).x)
            out = out + _ +"\n"
        return out

class Part:
    def __init__(self, vals):
        self.x = vals[0]
        self.y = vals[1]
        self.c = vals[2]
        self.exists = self.c != 0.0

    def __floordiv__(self, scalar):
        __truediv__(self, scalar)

    def __truediv__(self, scalar):
        return Part([self.x / scalar, self.y / scalar, self.c])

    @staticmethod
    def dist(part1, part2):
        return np.sqrt(np.square(part1.x - part2.x) + np.square(part1.y - part2.y))

In [19]:
import argparse
import glob
import json
import numpy as np
import os

from pose import Pose, Part, PoseSequence
from pprint import pprint



def parse_sequence(json_folder, output_folder):
    """Parse a sequence of OpenPose JSON frames and saves a corresponding numpy file.

    Args:
        json_folder: path to the folder containing OpenPose JSON for one video.
        output_folder: path to save the numpy array files of keypoints.

    """
    json_files = glob.glob(os.path.join(json_folder, '*.json'))
    json_files = sorted(json_files)

    num_frames = len(json_files)
    all_keypoints = np.zeros((num_frames, 18, 3))
    for i in range(num_frames):
        with open(json_files[i]) as f:
            json_obj = json.load(f)
            keypoints = np.array(json_obj['people'][0]['pose_keypoints'])
            all_keypoints[i] = keypoints.reshape((18, 3))
    
    output_dir = os.path.join(output_folder, os.path.basename(json_folder))
    np.save(output_dir, all_keypoints)


def load_ps(filename):
    """Load a PoseSequence object from a given numpy file.

    Args:
        filename: file name of the numpy file containing keypoints.
    
    Returns:
        PoseSequence object with normalized joint keypoints.
    """
    all_keypoints = np.load(filename)
#     print(all_keypoints)
    return PoseSequence(all_keypoints)




In [41]:
ps = load_ps('MyData/Data/'+"bicep_good_1.npy")
# ps = load_ps('poses_compressed/bicep/'+"bicep_good_1.npy")

poses = ps.poses
poses[0].relbow.x

[180.54422771 177.40993136 180.79347328 182.12070722 184.53030311
 181.5305779  180.42481886 179.7443601  182.9090251  178.02272643
 183.84151067 180.75234239 176.29400602 180.50224621 174.87669803
 169.64500529 183.10714585 182.64654161 183.12641604 176.93776816
 180.69947343 181.40573802 180.76828661 178.8016849  175.09346868
 178.57552812 172.80383866 178.148875   175.90557882 180.41241981
 188.34212066 182.44560603 186.32529432 185.14411216 184.27182912
 183.97455251 185.20445512 184.02766278 183.1929151  185.53882373
 183.3817386  181.82238046 186.34215386 184.03781326 189.02008334
 183.37983857 185.69284728 189.94472146 186.21327088 188.85763577
 192.05352856 189.58585752 181.15818028 176.21155256 165.35943477
 174.02258356 172.3265333  173.2557426  176.08699718 172.98902097
 169.08980472 168.54746073 166.85033274 169.40961862 173.24311107
 166.13169053 166.05134184 170.28504282 168.06088601 169.78969721
 170.65414657 169.73429674 164.80536414 170.17832236 172.7005861
 164.592902

0.5266051226522063